<a href="https://colab.research.google.com/github/opsifiz/AIB5_Emotion-Classification/blob/main/archive/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('drive/MyDrive/cleaned_dataset.csv', index_col=False)

In [ ]:
df

,text,status,from
0,oh my gosh,Anxiety,df1
1,trouble sleeping confused mind restless heart ...,Anxiety,df1
2,all wrong back off dear forward doubt stay in ...,Anxiety,df1
3,i have shifted my focus to something else but ...,Anxiety,df1
4,i am restless and restless it is been a month ...,Anxiety,df1
...,...,...,...
1093453,where can you go to get help for someone menta...,mentalillness,df3
1093454,i am rooster illusion ama,mentalillness,df3
1093455,i look again to make sure so one is near me an...,mentalillness,df3
1093456,nami peertopeer is a unique experiential learn...,mentalillness,df3


# Create Dataset for Model #1

In [ ]:
temp = df[df['status'].isin(['BPD', 'Anxiety', 'depression', 'mentalillness', 'bipolar', 'schizophrenia', 'Normal'])]
temp = temp.groupby('status').head(15000)

temp

,text,status,from
0,oh my gosh,Anxiety,df1
1,trouble sleeping confused mind restless heart ...,Anxiety,df1
2,all wrong back off dear forward doubt stay in ...,Anxiety,df1
3,i have shifted my focus to something else but ...,Anxiety,df1
4,i am restless and restless it is been a month ...,Anxiety,df1
...,...,...,...
1039331,low testosterone after discontinuing rispredon...,schizophrenia,df3
1039332,how did you finally accept your diagnosis i am...,schizophrenia,df3
1039333,constantly feel like i am in a competition wit...,schizophrenia,df3
1039334,has anyone switched over to an entirely differ...,schizophrenia,df3


In [ ]:
temp.to_csv('drive/MyDrive/train_dataset.csv', index=False)

#Create Dataset for Model #2

In [47]:
temp = df[df['status'].isin(['Anxiety', 'BPD', 'Normal', 'Stress', 'Suicidal', 'bipolar', 'depression', 'mentalillness', 'schizophrenia'])]
temp = temp.groupby('status').head(10000)
temp.drop(columns=['from'], inplace=True)
temp = temp.reset_index(drop=True)
temp

,text,status
0,oh my gosh,Anxiety
1,trouble sleeping confused mind restless heart ...,Anxiety
2,all wrong back off dear forward doubt stay in ...,Anxiety
3,i have shifted my focus to something else but ...,Anxiety
4,i am restless and restless it is been a month ...,Anxiety
...,...,...
82267,stuff my voices tell me they say that they hat...,schizophrenia
82268,bipolar and schizophrenia in remission after w...,schizophrenia
82269,nonpharmaceutical therapies hi i am someone wh...,schizophrenia
82270,got a job at baskin robbins what is your exper...,schizophrenia


In [ ]:
temp['status'].value_counts()

,count
status,
Anxiety,10000
Normal,10000
depression,10000
Suicidal,10000
bipolar,10000
schizophrenia,10000
BPD,10000
mentalillness,10000
Stress,2272


In [48]:
!pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.8 MB/s eta 0:00:00


In [50]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [54]:
import nlpaug.augmenter.word as naw
import pandas as pd

# Assume df is your original DataFrame
stress_df = temp[temp['status'] == 'Stress']

# Augmenter - Synonym replacement for demonstration
aug = naw.SynonymAug(aug_src='wordnet')

augmented_texts = []
for i in range(7728):
    original_text = stress_df['text'].sample(1).values[0]
    augmented_text = aug.augment(original_text)
    augmented_texts.append({'text': augmented_text, 'status': 'Stress'})

# Create a DataFrame from augmented texts
augmented_df = pd.DataFrame(augmented_texts)

# Combine with original
combined_df = pd.concat([temp, augmented_df], ignore_index=True)


In [67]:
combined_df

,text,status
0,oh my gosh,Anxiety
1,trouble sleeping confused mind restless heart ...,Anxiety
2,all wrong back off dear forward doubt stay in ...,Anxiety
3,i have shifted my focus to something else but ...,Anxiety
4,i am restless and restless it is been a month ...,Anxiety
...,...,...
89995,[i live in a high end building and i feel if s...,Stress
89996,[when you cannot remember the simplest things ...,Stress
89997,[i am in desperate need for someone online to ...,Stress
89998,[is this a psychological condition or scarce i...,Stress


In [70]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 8.7 MB/s eta 0:00:00


In [71]:
import contractions
import html
import re

In [69]:
#Clean Special Character
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = contractions.fix(text)
    text = html.unescape(text)
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Data Argment Stress Text untill 10,000 unique messages!

In [88]:
import nlpaug.augmenter.word as naw
import pandas as pd

# Assume df is your original DataFrame
stress_df = combined_df[combined_df['status'] == 'Stress']

# Augmenter - Synonym replacement for demonstration
aug = naw.SynonymAug(aug_src='wordnet')

augmented_texts = []
for i in range(15):
    original_text = stress_df['text'].sample(1).values[0]
    augmented_text = aug.augment(original_text)
    augmented_texts.append({'text': augmented_text, 'status': 'Stress'})

# Create a DataFrame from augmented texts
augmented_df = pd.DataFrame(augmented_texts)

# Combine with original
combined_df = pd.concat([combined_df, augmented_df], ignore_index=True)


In [89]:
combined_df['text'] = combined_df['text'].apply(clean_text)

In [96]:
combined_df.describe()

,text,status
count,90000,90000
unique,90000,9
top,is this a psychological condition or scarce in...,Anxiety
freq,1,10000


In [92]:
combined_df['status'].value_counts()

,count
status,
Anxiety,10000
Normal,10000
depression,10000
Suicidal,10000
Stress,10000
bipolar,10000
BPD,10000
schizophrenia,10000
mentalillness,10000


In [86]:
combined_df = combined_df.drop_duplicates(subset='text', keep='first')

In [94]:
combined_df.to_csv('drive/MyDrive/train_dataset2.csv', index=False)